## 2021: Week 35 - Picture Perfect

Data preppin' ideas really are all around! I was hanging some pictures the other week and I had some weird and wonderful sizes, so working out the perfect frames was a bit tiresome. If only I could use a data preppin' tool to speed up the process... 

### Input
We have 2 inputs this week:

1. Picture sizes 
![img](https://lh3.googleusercontent.com/-G8_8TINw7TY/YS591OZq5aI/AAAAAAAAA6o/n2-yPvhv2jUs30JKS4f2IZZ0NgHUmiarACLcBGAsYHQ/image.png)

2. Frame sizes
![img](https://lh3.googleusercontent.com/-AvDNsXSd9Bk/YS9tMox-bbI/AAAAAAAAA7I/9AE5n6GB91QV5xJuafIV6-NBdq67hymegCLcBGAsYHQ/image.png)

### Requirement
- Input the data
- Split up the sizes of the pictures and the frames into lengths and widths
    - Remember an inch is 2.54cm
- Frames can always be rotated, so make sure you know which is the min/max side
- See which pictures fit into which frames
- Work out the area of the frame vs the area of the picture and choose the frame with the smallest excess
- Output the data

### Output
![img](https://lh3.googleusercontent.com/-ccVraoOGt-I/YS6L0ikCBLI/AAAAAAAAA64/6NunkLPc7ywjecilgrlCmf_TFhzREMtWgCLcBGAsYHQ/image.png)

- 4 fields
    - Picture
    - Frame
    - Max Side
    - Min Side
- 14 rows (15 including headers)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
### Input the data

In [2]:
data = pd.read_excel("./data/Pictures Input.xlsx", sheet_name=[0, 1])

In [4]:
pictures = data[0].copy()
frames = data[1].copy()